In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
import requests
import pandas as pd

api_key = os.environ.get('riot_api_key')

def get_puuid(summonerId=None, gameName=None, tagLine=None, region='americas'):
    """Gets the puuid from a summonerId or riot_id and riot_tag

    Args:
        summonerId(str, opitional): Summoner ID, Defaults to None.
        gameName(str, opitional): Riot ID, Defaults to None. 
        tagLine(str, opitional): Riot Tag, Defaults to None. 
        region(str, opitional): Region, Defaults to 'americas'.

    returns: 
        str: puuid
    """
    if summonerId is not None: 
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = 'lol/summoner/v4/summoners/'
        print(root_url+endpoint+summonerId+'?api_key='+api_key)
        response = requests.get(root_url+endpoint+summonerId+'?api_key='+api_key)

        return response.json()['puuid']
    else:
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = f'riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}'
        response = requests.get(root_url+endpoint+'?api_key='+api_key)
        
        return response.json()['puuid']
    
def get_idtag_from_puuid(puuid=None):
    """Gets the riot_id and riot_tag from puuid

    Args:
        puuid(str, optional): puuid, Defaults to None.
    
    Returns:
        id(dict): Dictionary with riot_id and riot_tag.
    """
    
    root_url = 'https://americas.api.riotgames.com'
    endpoint = 'riot/account/v1/accounts/by-puuid/'

    response = requests.get(root_url+endpoint+puuid+'?api_key='+api_key)

    id = {
        'gameName': response.json()['gameName'],
        'tagLine': response.json()['tagLine']
    }

    return id

def get_ladder(top=300):
    """ Gets the top X players in soloQ.

    Args:
        top(int, optional) Number of players to return. Defaults to 300.
    Returns: 
        DataFrame: Returns a DataFrame of the top X players in soloq.
    """

    root_url = 'https://br1.api.riotgames.com/'
    challenger = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    grandmaster = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    master = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

    chall_response = requests.get(root_url + challenger + '?api_key=' + api_key)
    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)

    gm_df = pd.DataFrame()
    master_df = pd.DataFrame()
    if top > 300:    
        gm_response = requests.get(root_url + grandmaster + '?api_key=' + api_key)
        gm_df = pd.DataFrame(gm_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    if top > 1000:
        masters_response = requests.get(root_url + master + '?api_key=' + api_key)
        master_df = pd.DataFrame(masters_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    
    df = pd.concat([chall_df, gm_df, master_df]).reset_index(drop=True)[:top]

    df = df.reset_index().drop(columns=['rank']).rename(columns={'index': 'rank'})
    df['rank'] += 1

    return df

def get_match_history(region=None, puuid=None, start=0, count=20):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'/lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?start={start}&count={count}'

    response = requests.get(root_url+endpoint+query_params+'&api_key='+api_key)

    return response.json()

def get_match_data_from_id(region=None, match_Id=None):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'/lol/match/v5/matches/{match_Id}'
    print(root_url+endpoint+'?api_key='+api_key)
    response = requests.get(root_url+endpoint+'?api_key='+api_key)

    return response.json()

def process_match_json(match_json, puuid):              
    ## Architecture
    metadata = match_json['metadata']
    info = match_json['info']
    players = info['participants']    
    participants = metadata['participants']
    teams = info['teams']
    player = players[participants.index(puuid)]
    perks = player['perks']
    stats = perks['statPerks']
    styles = perks['styles']

    primary = styles[0]
    secondary = styles[1]

    side_dict = {
        100:'blue',
        200:'red'
    }

    ## Information
    match_Id = match_json['metadata']['matchId']

    game_creation = info['gameCreation']
    game_duration = info['gameDuration']
    game_start_timestamp = info['gameStartTimestamp']
    game_end_timestamp = info['gameEndTimestamp']
    patch = info['gameVersion']

    riot_id = player['riotIdGameName']
    riot_tag =player['riotIdTagline']
    summoner_id = player['summonerId']
    summoner_name = player['summonerName']

    side = side_dict[player['teamId']]
    win = player['win']

    champ_id = player['championId']
    champ_transform = player['championTransform']
    champ_level = player['champLevel']

    gold_earned = player['goldEarned']
    neutral_minion_killed = player['neutralMinionsKilled']
    total_minions_killed = player['totalMinionsKilled']
    total_ally_jungle_minions_killed = player['totalAllyJungleMinionsKilled']
    total_enemy_jungle_minions_killed = player['totalEnemyJungleMinionsKilled']

    kills = player['kills']
    assists = player['assists']
    deaths = player['deaths']
    first_blood = player['firstBloodKill']

    totel_damage_delt = player['totalDamageDealt']
    total_damage_shilded = player['totalDamageShieldedOnTeammates']
    total_damage_taken = player['totalDamageTaken']
    total_damage_healed = player['totalHealsOnTeammates']
    total_time_cc_dealt = player['totalTimeCCDealt']

    early_surrender = player['gameEndedInEarlySurrender']
    surrender = player['gameEndedInSurrender']

    item0 = player['item0']
    item1 = player['item1']
    item2 = player['item2']
    item3 = player['item3']
    item4 = player['item4']
    item5 = player['item5']
    item6 = player['item6']

    summoner_1_id = player['summoner1Id']
    summoner_2_id = player['summoner2Id']

    wards_placed = player['wardsPlaced']
    wards_killed = player['wardsKilled']
    vision_score = player['visionScore']

    #what is this? 
    detector_wards_placed = player['detectorWardsPlaced']
    role = player['role']

    defense = stats['defense']
    flex = stats['flex']
    offense = stats['offense']

    primary_style = primary['style']
    secondary_style = secondary['style']

    primary_keystone = primary['selections'][0]['perk']
    primary_perk_1 = primary['selections'][1]['perk']
    primary_perk_2 = primary['selections'][2]['perk']
    primary_perk_3 = primary['selections'][3]['perk']

    secondary_perk_1 = secondary['selections'][0]['perk']
    secondary_perk_2 = secondary['selections'][1]['perk']

    objective_stolen = player['objectivesStolen']
    objective_stolen_assists = player['objectivesStolenAssists']


    matchDF = pd.DataFrame({
        'match_Id': [match_Id],
        'participants': [participants],
        'game_creation': [game_creation],
        'game_duration': [game_duration],
        'game_start_timestamp': [game_start_timestamp],
        'game_end_timestamp': [game_end_timestamp],
        'patch': [patch],
        'puuid': [puuid],
        'riot_id': [riot_id],
        'riot_tag': [riot_tag],
        'summoner_id': [summoner_id],
        'summoner_name': [summoner_name],
        'win': [win],
        'champion': [champ_id],
        'champ_transform': [champ_transform],
        'champ_level': [champ_level],
        'kills': [kills],
        'deaths': [deaths],
        'assists':[assists],
        'summoner1_id': [summoner_1_id],
        'summoner2_id': [summoner_2_id],
        'gold_earned': [gold_earned],
        'total_minions_killed': [total_minions_killed],
        'total_neutral_minions_killed': [neutral_minion_killed],
        'total_ally_jungle_minions_killed': [total_ally_jungle_minions_killed],
        'total_enemy_jungle_minions_killed': [total_enemy_jungle_minions_killed],
        'early_surrender': [early_surrender],
        'surrender': [surrender],
        'first_blood': [first_blood],
        'objectives_stolen': [objective_stolen],
        'objective_stonlen_assists': [objective_stolen_assists],
        'total_damage_delt_champions': [totel_damage_delt],
        'total_damage_taken': [total_damage_taken],
        'total_damage_shilded_teammates': [total_damage_shilded],
        'total_heals_teammates': [total_damage_healed],
        'total_time_crowd_controlled': [total_time_cc_dealt],
        'vision_score': [vision_score],
        'wards_killed': [wards_killed],
        'wards_placed': [wards_placed],
        'control_wards_placed': [detector_wards_placed],
        'item0': [item0],
        'item1': [item1],
        'item2': [item2],
        'item3': [item3],
        'item4': [item4],
        'item5': [item5],
        'item6': [item6],
        'park_primary_row_1': [primary_perk_1],
        'park_primary_row_2': [primary_perk_2],
        'park_primary_row_3': [primary_perk_3],
        'park_secondary_row_1': [secondary_perk_1],
        'park_secondary_row_2': [secondary_perk_2],
        'perk_primary_style': [primary_style],
        'perk_secondary_style': [secondary_style],
        'park_shard_defense': [defense],
        'park_shard_flex': [flex],
        'park_shard_offense': [offense],
        
    })
    return matchDF

In [2]:
get_puuid(summonerId='z0px8f_1exj4jMA6rQ-7yAzzDnBkcxOIotfeg4ZpqH2sQA', region='br1')

https://br1.api.riotgames.com/lol/summoner/v4/summoners/z0px8f_1exj4jMA6rQ-7yAzzDnBkcxOIotfeg4ZpqH2sQA?api_key=RGAPI-6e45c473-1390-42c3-8075-ee0f76434f6d


'g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_FpBYYZ1JcKu3WvwilQnLNSem1QLskijibA'

In [3]:
get_ladder(20)

,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1,z0px8f_1exj4jMA6rQ-7yAzzDnBkcxOIotfeg4ZpqH2sQA,1751,279,221,True,False,False,False
1,2,mJeIYif9KVsFZ61unGVxqZcetiKwEi6bq0Whx_EQtqdnW5o,1712,144,79,True,False,False,False
2,3,jlrnYK1wjlFUH15V7p6The_r_AZo0NaogvVlFUba8PBW,1622,352,300,True,False,False,True
3,4,Oek5G0_0_if_tmyR6FIQ4qu7KELVj2vLeRfeGujPOuOahJ...,1605,168,116,True,False,False,False
4,5,wVaZVEksgw2JmmLiSrpmiL0TCoO6TT-guwxNzUERbhTi,1570,283,226,True,False,False,False
5,6,3CA_DtQKiypqHEX6OxgrlzgC5KSxkW32Np2UxW-myPfMmWc,1502,136,82,True,False,False,True
6,7,FtLOHPEuP08Y8ue6oWGZKtKmDaxFHNnxRaU-nP66JvIH7A,1450,305,259,True,False,False,False
7,8,9G-MWOyJb04DA856ylVQs-CmuTBtUqz_pyfCnDkaEcY_3k...,1450,148,99,True,False,False,False
8,9,A9O0dQheNkTC7edN1Nh2R0Dc5TeJzoK15dXgWjS4sRHB,1406,242,187,True,False,False,False
9,10,z5eJEznP5WX941tHbkMlfF4aKU8jySknRp1E7I-QAH-JiA,1375,158,110,True,False,False,False


In [4]:
get_match_history(region='americas', puuid='g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_FpBYYZ1JcKu3WvwilQnLNSem1QLskijibA', start=0, count=100)

['BR1_3030121902',
 'BR1_3030098770',
 'BR1_3030072982',
 'BR1_3030065544',
 'BR1_3029745275',
 'BR1_3029722809',
 'BR1_3029687777',
 'BR1_3029393496',
 'BR1_3029372895',
 'BR1_3029350175',
 'BR1_3029244970',
 'BR1_3029206012',
 'BR1_3028948996',
 'BR1_3028938104',
 'BR1_3028927359',
 'BR1_3028917296',
 'BR1_3028711352',
 'BR1_3028673783',
 'BR1_3028616419',
 'BR1_3028576307',
 'BR1_3028553164',
 'BR1_3028528315',
 'BR1_3028494113',
 'BR1_3028473358',
 'BR1_3028439794',
 'BR1_3028391764',
 'BR1_3028366494',
 'BR1_3028088453',
 'BR1_3028062393',
 'BR1_3028039413',
 'BR1_3028006921',
 'BR1_3027988415',
 'BR1_3027941566',
 'BR1_3027905826',
 'BR1_3027716713',
 'BR1_3027615742',
 'BR1_3027600401',
 'BR1_3027574047',
 'BR1_3027342500',
 'BR1_3027317960',
 'BR1_3027274063',
 'BR1_3027197819',
 'BR1_3027166407',
 'BR1_3027148347',
 'BR1_3027137131',
 'BR1_3026889730',
 'BR1_3026867733',
 'BR1_3026820665',
 'BR1_3026794912',
 'BR1_3026718779',
 'BR1_3026706125',
 'BR1_3026685045',
 'BR1_302666

In [5]:
match_ids = get_match_history(region='americas', puuid='g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_FpBYYZ1JcKu3WvwilQnLNSem1QLskijibA')

df = pd.DataFrame()
for match_id in match_ids:
    game = get_match_data_from_id(region='americas', match_Id=match_id)
    matchDF = process_match_json(game, puuid='g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_FpBYYZ1JcKu3WvwilQnLNSem1QLskijibA')
    
    df = pd.concat([df, matchDF])

https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3030121902?api_key=RGAPI-6e45c473-1390-42c3-8075-ee0f76434f6d
https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3030098770?api_key=RGAPI-6e45c473-1390-42c3-8075-ee0f76434f6d
https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3030072982?api_key=RGAPI-6e45c473-1390-42c3-8075-ee0f76434f6d
https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3030065544?api_key=RGAPI-6e45c473-1390-42c3-8075-ee0f76434f6d
https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3029745275?api_key=RGAPI-6e45c473-1390-42c3-8075-ee0f76434f6d
https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3029722809?api_key=RGAPI-6e45c473-1390-42c3-8075-ee0f76434f6d
https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3029687777?api_key=RGAPI-6e45c473-1390-42c3-8075-ee0f76434f6d
https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3029393496?api_key=RGAPI-6e45c473-1390-42c3-8075-ee0f76434f6d
https://americas

In [6]:
df

,match_Id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,...,park_primary_row_1,park_primary_row_2,park_primary_row_3,park_secondary_row_1,park_secondary_row_2,perk_primary_style,perk_secondary_style,park_shard_defense,park_shard_flex,park_shard_offense
0,BR1_3030121902,[XfiywZL39WkuTU32ubTVmqBWvBlIluAjpxRQH1o6PoeRc...,1732733282500,1351,1732733341597,1732734692715,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,9111,9104,8017,8143,8135,8000,8100,5011,5008,5005
0,BR1_3030098770,[CZI1GrKo00WDgQDilrOj2l295zkx6zhZhs4zzAhL1v7p-...,1732730131559,2003,1732730202599,1732732206316,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,9111,9104,8017,8143,8135,8000,8100,5011,5008,5005
0,BR1_3030072982,[eQX5YFUkXoy_1NN-RYva_52buPfPde-3qHrD9t9dytom2...,1732726656440,1568,1732726700416,1732728269187,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,8009,9104,8017,8345,8347,8000,8300,5011,5008,5005
0,BR1_3030065544,[CXcpbO3wmEc-r1KFm1d7PzWQtpfFU2Dcm7AV8PnTZaH2V...,1732725159665,961,1732725232180,1732726193941,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,8009,9104,8017,8304,8345,8000,8300,5011,5008,5005
0,BR1_3029745275,[cuVj_EdCMK6dlH6OG67ZL-ueA-MUZXnrwrlziJXX1FCXG...,1732664160759,2348,1732664193643,1732666542344,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,8009,9104,8017,8304,8345,8000,8300,5011,5008,5005
0,BR1_3029722809,[yOvDDEJPXj_AXwxJSIf7EEyuiIzzwV8_hNEX60cqJg6cz...,1732662370399,1356,1732662406925,1732663762780,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,9111,9104,8017,8429,8453,8000,8400,5011,5008,5005
0,BR1_3029687777,[HaZjwnBChtxUY6YLlpJ4eI4Q8lFhD8jTLUwH9zkh0_X-8...,1732659719397,1488,1732659746975,1732661235245,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,8009,9103,8017,8345,8304,8000,8300,5011,5008,5005
0,BR1_3029393496,[WRPJROwvZUTQgGiCoSSmVGJK6FZASAo9C89distiVBH9q...,1732593096980,1692,1732593170551,1732594862957,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,8009,9104,8017,8304,8345,8000,8300,5011,5008,5005
0,BR1_3029372895,[jp5grFdjV7jBI35sHYSvgHzu_uNndNUDrXA_IooS_UfPF...,1732590446070,1613,1732590475041,1732592088801,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,8009,9104,8017,8304,8345,8000,8300,5011,5008,5005
0,BR1_3029350175,[eQX5YFUkXoy_1NN-RYva_52buPfPde-3qHrD9t9dytom2...,1732587886298,1801,1732587912439,1732589714129,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,8009,9104,8017,8304,8345,8000,8300,5011,5008,5005


In [7]:
temp_dict = {
    8009: 'Perk 1',
    9111: 'Perk 2',
}

In [8]:
df.replace(temp_dict)

,match_Id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,...,park_primary_row_1,park_primary_row_2,park_primary_row_3,park_secondary_row_1,park_secondary_row_2,perk_primary_style,perk_secondary_style,park_shard_defense,park_shard_flex,park_shard_offense
0,BR1_3030121902,[XfiywZL39WkuTU32ubTVmqBWvBlIluAjpxRQH1o6PoeRc...,1732733282500,1351,1732733341597,1732734692715,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Perk 2,9104,8017,8143,8135,8000,8100,5011,5008,5005
0,BR1_3030098770,[CZI1GrKo00WDgQDilrOj2l295zkx6zhZhs4zzAhL1v7p-...,1732730131559,2003,1732730202599,1732732206316,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Perk 2,9104,8017,8143,8135,8000,8100,5011,5008,5005
0,BR1_3030072982,[eQX5YFUkXoy_1NN-RYva_52buPfPde-3qHrD9t9dytom2...,1732726656440,1568,1732726700416,1732728269187,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Perk 1,9104,8017,8345,8347,8000,8300,5011,5008,5005
0,BR1_3030065544,[CXcpbO3wmEc-r1KFm1d7PzWQtpfFU2Dcm7AV8PnTZaH2V...,1732725159665,961,1732725232180,1732726193941,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Perk 1,9104,8017,8304,8345,8000,8300,5011,5008,5005
0,BR1_3029745275,[cuVj_EdCMK6dlH6OG67ZL-ueA-MUZXnrwrlziJXX1FCXG...,1732664160759,2348,1732664193643,1732666542344,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Perk 1,9104,8017,8304,8345,8000,8300,5011,5008,5005
0,BR1_3029722809,[yOvDDEJPXj_AXwxJSIf7EEyuiIzzwV8_hNEX60cqJg6cz...,1732662370399,1356,1732662406925,1732663762780,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Perk 2,9104,8017,8429,8453,8000,8400,5011,5008,5005
0,BR1_3029687777,[HaZjwnBChtxUY6YLlpJ4eI4Q8lFhD8jTLUwH9zkh0_X-8...,1732659719397,1488,1732659746975,1732661235245,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Perk 1,9103,8017,8345,8304,8000,8300,5011,5008,5005
0,BR1_3029393496,[WRPJROwvZUTQgGiCoSSmVGJK6FZASAo9C89distiVBH9q...,1732593096980,1692,1732593170551,1732594862957,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Perk 1,9104,8017,8304,8345,8000,8300,5011,5008,5005
0,BR1_3029372895,[jp5grFdjV7jBI35sHYSvgHzu_uNndNUDrXA_IooS_UfPF...,1732590446070,1613,1732590475041,1732592088801,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Perk 1,9104,8017,8304,8345,8000,8300,5011,5008,5005
0,BR1_3029350175,[eQX5YFUkXoy_1NN-RYva_52buPfPde-3qHrD9t9dytom2...,1732587886298,1801,1732587912439,1732589714129,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Perk 1,9104,8017,8304,8345,8000,8300,5011,5008,5005


In [9]:
perk = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json'
perk_styles = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perkstyles.json'

perk_json = requests.get(perk).json()
perk_styles_json = requests.get(perk_styles).json()

In [10]:
perk_json

[{'id': 8369,
  'name': 'First Strike',
  'majorChangePatchVersion': '11.23',
  'tooltip': 'Attacks or abilities against an enemy champion within @GraceWindow.2@s of entering champion combat grant @GoldProcBonus@ gold and <b>First Strike</b> for @Duration@ seconds, causing you to deal <truedamage>@DamageAmp*100@%</truedamage> extra <truedamage>damage</truedamage> against champions, and granting <gold>{{ Item_Melee_Ranged_Split }}</gold> of that damage dealt as <gold>gold</gold>.<br><br>Cooldown: <scaleLevel>@Cooldown@</scaleLevel>s<br><hr><br>Damage Dealt: @f1@<br>Gold Gained: @f2@',
  'shortDesc': 'When you initiate champion combat, deal 7% extra damage for 3 seconds and gain gold based on damage dealt.',
  'longDesc': 'Attacks or abilities against an enemy champion within 0.25s of entering champion combat grant 10 gold and <b>First Strike</b> for 3 seconds, causing you to deal <truedamage>7%</truedamage> extra <truedamage> damage</truedamage> against champions, and granting <gold>50%

In [11]:
def json_extract(obj, key):
    arr = []

    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key and v not in arr:
                    arr.append(v)
                elif isinstance(v, (dict, list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)

        return arr
    
    values = extract(obj, arr, key)
    return values

In [12]:
perk_ids = json_extract(perk_json, 'id')
perk_names = json_extract(perk_json, 'name')

perk_dict = dict(map(lambda i, j: (i, j), perk_ids, perk_names))

perk_style_ids = json_extract(perk_styles_json, 'id')
perk_style_names = json_extract(perk_styles_json, 'name')\

perk_style_ids_int = [i for i in perk_style_ids if isinstance(i, int)]

perk_styles_dict = dict(map(lambda i, j: (i, j), perk_style_ids_int, perk_style_names[:len(perk_style_ids_int)]))

In [13]:
perk_styles_dict

{8400: 'Resolve',
 8100: 'Domination',
 8000: 'Precision',
 8200: 'Sorcery',
 8300: 'Inspiration'}

In [14]:
df.replace(perk_dict).replace(perk_styles_dict)

,match_Id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,...,park_primary_row_1,park_primary_row_2,park_primary_row_3,park_secondary_row_1,park_secondary_row_2,perk_primary_style,perk_secondary_style,park_shard_defense,park_shard_flex,park_shard_offense
0,BR1_3030121902,[XfiywZL39WkuTU32ubTVmqBWvBlIluAjpxRQH1o6PoeRc...,1732733282500,1351,1732733341597,1732734692715,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Triumph,Legend: Alacrity,Cut Down,Sudden Impact,Treasure Hunter,Precision,Domination,Health,Adaptive Force,Attack Speed
0,BR1_3030098770,[CZI1GrKo00WDgQDilrOj2l295zkx6zhZhs4zzAhL1v7p-...,1732730131559,2003,1732730202599,1732732206316,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Triumph,Legend: Alacrity,Cut Down,Sudden Impact,Treasure Hunter,Precision,Domination,Health,Adaptive Force,Attack Speed
0,BR1_3030072982,[eQX5YFUkXoy_1NN-RYva_52buPfPde-3qHrD9t9dytom2...,1732726656440,1568,1732726700416,1732728269187,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Presence of Mind,Legend: Alacrity,Cut Down,Biscuit Delivery,Cosmic Insight,Precision,Inspiration,Health,Adaptive Force,Attack Speed
0,BR1_3030065544,[CXcpbO3wmEc-r1KFm1d7PzWQtpfFU2Dcm7AV8PnTZaH2V...,1732725159665,961,1732725232180,1732726193941,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Presence of Mind,Legend: Alacrity,Cut Down,Magical Footwear,Biscuit Delivery,Precision,Inspiration,Health,Adaptive Force,Attack Speed
0,BR1_3029745275,[cuVj_EdCMK6dlH6OG67ZL-ueA-MUZXnrwrlziJXX1FCXG...,1732664160759,2348,1732664193643,1732666542344,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Presence of Mind,Legend: Alacrity,Cut Down,Magical Footwear,Biscuit Delivery,Precision,Inspiration,Health,Adaptive Force,Attack Speed
0,BR1_3029722809,[yOvDDEJPXj_AXwxJSIf7EEyuiIzzwV8_hNEX60cqJg6cz...,1732662370399,1356,1732662406925,1732663762780,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Triumph,Legend: Alacrity,Cut Down,Conditioning,Revitalize,Precision,Resolve,Health,Adaptive Force,Attack Speed
0,BR1_3029687777,[HaZjwnBChtxUY6YLlpJ4eI4Q8lFhD8jTLUwH9zkh0_X-8...,1732659719397,1488,1732659746975,1732661235245,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Presence of Mind,Legend: Bloodline,Cut Down,Biscuit Delivery,Magical Footwear,Precision,Inspiration,Health,Adaptive Force,Attack Speed
0,BR1_3029393496,[WRPJROwvZUTQgGiCoSSmVGJK6FZASAo9C89distiVBH9q...,1732593096980,1692,1732593170551,1732594862957,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Presence of Mind,Legend: Alacrity,Cut Down,Magical Footwear,Biscuit Delivery,Precision,Inspiration,Health,Adaptive Force,Attack Speed
0,BR1_3029372895,[jp5grFdjV7jBI35sHYSvgHzu_uNndNUDrXA_IooS_UfPF...,1732590446070,1613,1732590475041,1732592088801,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Presence of Mind,Legend: Alacrity,Cut Down,Magical Footwear,Biscuit Delivery,Precision,Inspiration,Health,Adaptive Force,Attack Speed
0,BR1_3029350175,[eQX5YFUkXoy_1NN-RYva_52buPfPde-3qHrD9t9dytom2...,1732587886298,1801,1732587912439,1732589714129,14.23.636.9832,g1ys191L5OIm4UDCD7SoTd9rT-kUWtNzqtbA_Xz-s0c_Fp...,Brance,Branc,...,Presence of Mind,Legend: Alacrity,Cut Down,Magical Footwear,Biscuit Delivery,Precision,Inspiration,Health,Adaptive Force,Attack Speed
